<a href="https://colab.research.google.com/github/ucfilho/raianars_adjust_RTC/blob/master/RTC_adjust_v_08_mar_08_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x):
  global Trocas,D, tempInicial, tempAtual 
  cols=len(tempInicial) 
  Ind_I=np.zeros((D,Trocas)).astype(np.int64)
  Ind_J=np.zeros((D,Trocas)).astype(np.int64)
  Tin=np.zeros((D,Trocas))
  Tout=np.zeros((D,Trocas))
  deltaT=np.zeros((D,Trocas))
  for i in range(D):
    for j in range(Trocas):
      Ind_I[i,j]=round(x[i,j])
      Ind_J[i,j]=round(x[i,j+Trocas])
      deltaT[i,j]=x[i,j+2*Trocas] 
    for jj in range(cols):
      Tin[i,jj]=tempInicial[jj]
      Tout[i,jj]=tempAtual[jj]
  return Ind_I,Ind_J,deltaT, Tin, Tout

In [0]:
#def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):
def Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin,Tout):
#def Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,tempInicial,tempAtual):
  global Trocas,D
  # cols=len(mCp)
  # i represent hot stream  and j represent cold stream 
  # k represent sequential heat exchanges
  cargaTermica=np.zeros((D,Trocas))
  contador=0
  cont_uso=0
  erro=np.zeros((D)) # valor erro se torna erro=1 quando ha algum erro e vai penalizar
  for ki in range(D):
    for kj in range(Trocas):
      k=kj
      i=Ind_I[ki,kj]
      j=Ind_J[ki,kj]
      '''
      print('=================')
      print(i,j)
      print('mCp[i]',mCp[i])
      print('deltaT[ki,kj]',deltaT[ki,kj])
      '''
      cargaTermica[ki,kj]=mCp[i]*deltaT[ki,kj]
      tempSaidaQ=Tin[ki,i]+deltaT[ki,kj]
      tempSaidaF=-cargaTermica[ki,kj]/(mCp[j])+Tout[ki,j]
      flag=0 # flag : indica que nao entra no teste logico
      # usei flag no lugar q ifs pq a identacao ficou melhor assim
      if(Tin[ki,i] > Tin[ki,j]): #Tquente maior Tfria: vaores atuais
        if(tempSaidaQ > tempSaidaF): #Tquente maior Tfria: saidas
          # if(contador < cols):
          flag=1
            
      if(flag>0):
        Tout[ki,i]=tempSaidaQ
        Tout[ki,j]=tempSaidaF
      else:
        erro[ki]=1


  return Tout,cargaTermica,erro

In [0]:
def Area(U,Tin,Tout,Ind_I,Ind_J,deltaT):

  global Trocas,mCp,D
  
  Area=[]

  for ki in range(D):
    Area_Particula=[]
    for kj in range(Trocas):
      i=Ind_I[ki,kj]
      j=Ind_J[ki,kj]
      '''
      print('================')
      print(i,j)
      print('mCp[i]',mCp[i])
      print('deltaT[ki,kj]',deltaT[ki,kj])
      print('ki',ki)
      print(Tout)
      print('Tout[ki,j]',Tout[ki,j])
      '''
      Carga=mCp[i]*deltaT[ki,kj]
      tempSaidaF=Tout[ki,j]
      tempSaidaQ=Tout[ki,i]
      deltaT1=Tin[ki,i]-tempSaidaF
      deltaT2=tempSaidaQ-Tin[ki,j]
      try:
        A=(deltaT1-deltaT2)
        B=np.log((deltaT1/deltaT2))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
  
      if(deltaTMediaLog==1e99):
        AreaCalc=1e99
      else:
        AreaCalc=abs(Carga/(U*deltaTMediaLog))

      if(Tout[ki,i]==Tin[ki,i]):
            AreaCalc=0

      if(Tout[ki,j]==Tin[ki,j]):
        AreaCalc=0
            
      Area_Particula.append(AreaCalc)
      
    Area.append(Area_Particula)

  return Area

In [0]:
def Utilidades(tempAlvo,Tout,mCp):

  cols=len(mCp)
  cargaUQ=0
  cargaUF=0
  
  '''
  caso I cita TUQin=553 e TQout=552, TUFin=303 TUFout=353
  '''
  
  TUQin=553
  TUQout=552
  TUFin=303 
  TUFout=353

  
  Areas=[]

  for ki in range(D):
    areaTrocador=[]
    for j in range(cols):
      delta=tempAlvo[j]-Tout[ki,j]
      if(delta>0):#usa UQ
        cargaUQ=cargaUQ+mCp[j]*delta
        deltaT1=(TUQin-tempAlvo[j])
        deltaT2=(TUQout-Tout[ki,j])

        try:
          A=deltaT1-deltaT2
          B=np.log(deltaT1/deltaT2)
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99

        if(deltaTMediaLog==1e99):
          areaTrocador.append(1e99)
        else:
          areaTrocador.append((abs(mCp[j]*(delta)/(U*deltaTMediaLog))))

      elif(delta==0):#nao usa UF e UQ
        deltaTMediaLog=0
        areaTrocador.append(0) 

      else:#usa UF
        deltaT1=(Tout[ki,j]-TUFout)
        deltaT2=(tempAlvo[j]-TUFin)     
        cargaUF=cargaUF+mCp[j]*(-delta)

        #UF -> água resfriada -> entra a 303K e sai a 353K
        try:
          A=deltaT1-deltaT2
          B=np.log(deltaT1/deltaT2)
          deltaTMediaLog=A/B
        except:
          deltaTMediaLog=1e99
        
        if(deltaTMediaLog==1e99):
          areaTrocador.append(1e99)
        else:
          areaTrocador.append((abs(mCp[j]*(-delta)/(U*deltaTMediaLog))))

    Areas.append(areaTrocador)
  
  # print(Areas)

  return Areas, cargaUF,cargaUQ 
  # areaTrocador contem a soma das areas
  # Trocadores a area associada a cada corrente
  # cargaUF e cargaUQ kW gasto

In [0]:
def CustoProcesso(Area,Utilidades,cargaUF,cargaUQ):
  global coefCustoUQ,coefCustoUF,D

  cost=np.zeros((D))

  for i in range(D):
    areaTrocadores=Area[i]
    areaUtilidades=Utilidades[i]
    costTrocadores=0
    for it in areaTrocadores:
      for wii in areaUtilidades:
        if(it<1e99):
          if(wii<1e99):
            costTrocadores=costTrocadores+300*(it**0.5+wii**0.5)
          else:
            costTrocadores=1e99
            break
          
    costUQ=coefCustoUQ*cargaUQ
    costUF=coefCustoUF*cargaUF

    if(costTrocadores==1e99):
      cost[i]=1e99
    else:
      cost[i]=costUQ+costUF+costTrocadores
      
  return cost

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  # global mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF
  global mCp,tempAtual,tempAlvo,tempInicial,U,coefCustoUQ,coefCustoUF
  global D
  
  funr=np.zeros((D))


  cols=len(mCp)
  Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
  tempAtual,Carga,Erro=Temperatura_Carga(mCp,Ind_I,Ind_J,deltaT,Tin,Tout)
  areaTrocadores =Area(U,Tin,Tout,Ind_I,Ind_J,deltaT)
  areaUtilidade, cargaUF,cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)
  cost=CustoProcesso(areaTrocadores,areaUtilidade,cargaUF,cargaUQ)
  funr=cost
 
  #print(Erro)
  #print(funr)

  for k in range(D):
    if(Erro[k]>0):
      funr[k]=1e99

  #funr = np.where(np.isnan(cost), 1e99, cost)

  return funr,Erro

In [0]:
#********************PROGRAMA PRINCIPAL*************************

NPAR=50 #Número de partículas
ITE=5000 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

MAX=[0,0,0,0,0,0,1,1,1,1,1,1]
MIN=[-300,-300,-300,-300,-300,-300,0,0,0,0,0,0]
mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453,393,523,533] #[K]
tempAtual=np.copy(tempInicial) 
#tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]

coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

bounds = []
for i in range(12):
  bounds.append((MAX[i],MIN[i]))



In [0]:
#MAX=[0,0,0,0,0,0,1,1,1,1,1,1]
#MIN=[-300,-300,-300,-300,-300,-300,0.3,0.3,0.3,0.3,0.3,0.3]
Trocas=4

D=10

cols=Trocas*3 # tres linhas i,j,deltaT
MAX=np.zeros((cols))
MIN=np.zeros((cols))

for k in range(cols):
  if(k>(Trocas*2-1)):
    MIN[k]=-100
  else:
    MAX[k]=(Trocas-1)

x=np.zeros((D,cols))

for i in range(D):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]




In [11]:
print(x)
print(x.shape)

[[  2.62374877   1.53710101   1.94208892   0.85621075   1.91893368
    2.57823134   2.01073823   1.07003459 -25.289477   -46.85716257
  -56.85438839 -69.61233336]
 [  0.86631006   1.53830208   2.61147593   1.61302523   1.28493699
    0.15745862   1.98010584   2.68504935 -74.75127778 -94.80333223
  -70.51335188 -79.68915509]
 [  2.21974472   0.48949875   1.07257274   0.72907941   2.95444732
    1.60988951   0.15252036   0.95427326 -87.28946227 -33.89815617
  -54.17642072 -54.29158911]
 [  1.27380913   2.30210716   1.03562376   1.20621397   1.40026768
    2.14198682   2.01338178   2.8529727  -84.57710762 -47.63292667
  -75.59371776 -72.19259964]
 [  2.43905042   0.54979614   1.58816942   2.42514872   0.66501012
    2.98821222   0.24296488   2.3479559  -39.85962231 -24.74760258
  -88.68729776 -94.49469644]
 [  2.36675117   1.74532007   1.80133433   2.79800557   1.08727386
    1.1798568    2.62518579   2.77328583 -25.31757196 -37.17141537
  -97.52228398  -3.00320399]
 [  0.46733866   2.796

In [12]:

Custo, Erro=FUN(x)
print(Custo)
print(Erro)

[1.e+99 1.e+99 1.e+99 1.e+99 1.e+99 1.e+99 1.e+99 1.e+99 1.e+99 1.e+99]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in log


In [13]:
print(MAX)

[3. 3. 3. 3. 3. 3. 3. 3. 0. 0. 0. 0.]


In [0]:
# print(x)

In [0]:

x=np.zeros((D,cols))
MAX=np.array([3,2,3,2,0,1,1,1,0,0,0,0])
MIN=np.array([3,2,3,2,0,1,1,1,-100,-100,-100,-100])
for i in range(D):
  for j in range(cols):
    r=np.random.random()
    x[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

In [16]:
print(x)
print(x.shape)

[[  3.           2.           3.           2.           0.
    1.           1.           1.         -97.04508857 -40.15423991
  -98.88303571 -19.12014211]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -86.1370251   -0.39789745
  -35.43163826 -63.68796864]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -61.16822275 -57.22345921
  -35.77239459 -80.28797785]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -78.30860741 -20.2570613
   -7.96396042 -93.52946711]
 [  3.           2.           3.           2.           0.
    1.           1.           1.          -1.23340481 -32.44489914
  -18.2369369  -59.44376104]
 [  3.           2.           3.           2.           0.
    1.           1.           1.         -27.71994639 -62.15850764
  -65.64397817 -16.01041634]
 [  3.           2.           3.           2.           0.
    1.      

In [0]:
tempInicial=[453,393,523,533] #[K]
tempAtual=np.copy(tempInicial) 


In [18]:
Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x) # transforma x em matriz logica e troca
print(Ind_I)


[[3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]
 [3 2 3 2]]


In [19]:
print(Ind_J)


[[0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]
 [0 1 1 1]]


In [20]:
Custo, Erro=FUN(x)
print(Custo)
print(Erro)

[1.00000000e+99 1.00000000e+99 1.00000000e+99 1.00000000e+99
 1.00000000e+99 1.00000000e+99 1.00000000e+99 4.46159004e+05
 1.00000000e+99 1.00000000e+99]
[1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


In [21]:
print(tempAtual)

[[453.         437.45578651 503.87985789 533.        ]
 [453.         446.44588048 522.60210255 497.56836174]
 [453.         489.5761863  465.77654079 497.22760541]
 [453.         420.13873661 502.7429387  525.03603958]
 [453.92505361 444.68907971 490.55510086 514.7630631 ]
 [473.78995979 451.62669298 506.98958366 505.28005361]
 [453.         421.06978293 485.57362276 533.        ]
 [473.54629781 459.46575816 518.65339559 499.69307168]
 [453.         448.67628952 522.16808153 533.        ]
 [453.         466.21141092 523.         484.19239272]]


In [22]:
# caso I: parecido, teste 
#Ind_x=[4,3,4,3]
#Ind_y=[1,2,2,2]
#deltaT=[-21,-75,-21,-27]
D=1
xtest=np.array([3,2,3,2,0,1,1,1,-21,-75,-21,-27]).reshape(1,12)
Ind_I,Ind_J,deltaT, Tin, Tout=buildMatriz(x)
print(Ind_I)
Custo, Erro=FUN(xtest)
print(Custo)
print(Erro)

ValueError: ignored

In [0]:
print(xtest[0,10])
print(xtest)